<a href="https://colab.research.google.com/github/alex-smith-uwec/CS491/blob/main/SentenceExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
my_path="/content/drive/MyDrive/CS491/Data/"

Some pre-processing: download novel from Gutenberg, then remove preamble and postamble.

In [3]:
title = 'The Golden Bowl'
tit='Bowl'
author = 'Henry James'
url = 'https://www.gutenberg.org/cache/epub/4264/pg4264.txt'
path = my_path


In [4]:
import os

In [5]:
# Check if the file is stored locally
filename = my_path+ title
if os.path.isfile(filename) and os.stat(filename).st_size != 0:
        print("{title} file already exists".format(title=title))
        with open(filename, 'r') as f:
            raw = f.read()

else:
    print("{title} file does not already exist. Grabbing from Project Gutenberg".format(title=title))
    response = request.urlopen(url)
    raw = response.read().decode('utf-8-sig')
    print("Saving {title} file".format(title=title))
    with open(filename, 'w') as outfile:
        outfile.write(raw)

The Golden Bowl file already exists


In the file browser to the left, open the book. The text should (might?) appear in a cell on the right. Delete preamble and postamble, then save.

In [6]:
# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t"]:
   raw = raw.replace(char, " ")

with open(filename, 'w') as outfile:
    outfile.write(raw)

In [7]:
print(f"first characters are: {raw[:50]}")
print(f"last characters are: {raw[-50:]}")

first characters are:        The Prince had always liked his London, whe
last characters are: d of them, she buried her own in his breast.      


# Onwards to sentences

In [8]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
with open(my_path+title, 'r') as file:
    all_sentences=sent_tokenize(file.read())#.lower())

In [10]:
len(all_sentences)

6953

In [11]:
import random

In [13]:
random.choice(all_sentences)

'That hadn’t been at all my idea, but she acted before I understood.'

In [14]:
all_sentences[0:2]

['       The Prince had always liked his London, when it had come to him; he was one of the modern Romans who find by the Thames a more convincing image of the truth of the ancient state than any they have left by the Tiber.',
 'Brought up on the legend of the City to which the world paid tribute, he recognised in the present London much more than in contemporary Rome the real dimensions of such a case.']

In [22]:
temp=random.choice(all_sentences)
print(temp)

But that’s exactly the ground of my question.


# Tagging and parsing
We experiment with average_perceptron_tagger, pos_tag, grammar, RegexpParser

In [23]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [81]:
# Menu of sentences to be tagged
# sentence = temp #see above, from novel.
# sentence="See Spot run."
sentence="The quick brown fox jumped over the lazy dog."
# sentence = "At eight o'clock on Thursday morning Arthur didn't feel very good."
# sentence='They can fish'#Interesting to see how this will get parsed!
print(sentence)

The quick brown fox jumped over the lazy dog.


In [83]:
# Tokenizing and tagging
tokens = nltk.word_tokenize(sentence)
tagged = nltk.pos_tag(tokens)

tagged

[('The', 'DT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'NN'),
 ('jumped', 'VBD'),
 ('over', 'IN'),
 ('the', 'DT'),
 ('lazy', 'JJ'),
 ('dog', 'NN'),
 ('.', '.')]

See [Penntree bank info](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) for part of speech tags.



In [ ]:
##This will show details about POS tags
# nltk.download('tagsets')
# nltk.help.upenn_tagset()

In [84]:
# Some possible grammars to create parse tree

# grammar=" " #empty grammar
# grammar = "NP: {<JJ><NN>}"
# grammar = "NP: {<DT>?<JJ>*<NN>}"

grammar = """
    NP: {<DT>?<JJ>*<NN>}                      # Noun Phrase
    VP: {<VB.*><NP|PP>+}                      # Verb Phrase
    PP: {<IN><NP>}                            # Prepositional Phrase
    ADJP: {<RB>*<JJ>}                         # Adjective Phrase
    CS: {<NP><VP><CC><NP><VP>}                # Compound Sentence
    NER: {<NNP>+}                             # Named Entity Recognition
"""


In [85]:
# Create tree
parser = nltk.RegexpParser(grammar)
tree = parser.parse(tagged)
print(tree)

(S
  (NP The/DT quick/JJ brown/NN)
  (NP fox/NN)
  jumped/VBD
  (PP over/IN (NP the/DT lazy/JJ dog/NN))
  ./.)
